In [2]:
import yfinance as yf
import pandas as pd

# Definisci il periodo di tempo
start_date = '2013-01-01'
end_date = '2023-01-01'

# Scarica i dati per Tesla
tsla = yf.download('TSLA', start=start_date, end=end_date)
# Scarica i dati per Coca-Cola
ko = yf.download('KO', start=start_date, end=end_date)

def calculate_volatility(data):
    # Calcola i rendimenti giornalieri
    data['Daily_Return'] = data['Close'].pct_change()
    # Calcola la volatilità come la deviazione standard dei rendimenti giornalieri
    data['Volatility'] = data['Daily_Return'].rolling(window=30).std() * (252**0.5)
    return data

tsla = calculate_volatility(tsla)
ko = calculate_volatility(ko)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import pandas as pd

# Definisci il periodo di tempo
start_date = '2013-01-01'
end_date = '2023-01-01'

# Scarica i dati per Tesla (TSLA)
tsla = yf.download('TSLA', start=start_date, end=end_date)
# Scarica i dati per Coca-Cola (KO)
ko = yf.download('KO', start=start_date, end=end_date)

# Funzione per creare tabelle annuali con prezzi di chiusura
def create_annual_close_table(data, asset_name):
    # Rimuovi le osservazioni con valori nulli o NaN
    data = data.dropna(subset=['Close'])
    years = data.index.year.unique()
    annual_data = []

    for year in years:
        yearly_data = data[data.index.year == year]
        close_prices = yearly_data['Close'].tolist()
        annual_data.append([year] + close_prices)

    annual_df = pd.DataFrame(annual_data)
    annual_df.columns = ['Year'] + [f'Day_{i+1}' for i in range(len(annual_df.columns) - 1)]
    return annual_df

# Creazione delle tabelle annuali per i prezzi di chiusura
tsla_close_annual = create_annual_close_table(tsla, 'TSLA')
ko_close_annual = create_annual_close_table(ko, 'KO')

# Esporta le tabelle annuali in formato CSV
tsla_close_annual.to_csv('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/data_tsla.csv', index=False)
ko_close_annual.to_csv('/Users/roberto/Desktop/UNI 3^ ANNO/tesi/dataset/data_kk.csv', index=False)

print("Tabelle annuali dei prezzi di chiusura create e salvate in file CSV separati con successo!")


Ora , tramite apposita funzione in matlab ho creato spettrogrammi per rappresentare l'informazione sotto forma di segnale per ciascuna stock, per ogni anno.. cosi ho creato molteplici immagini per addestrare il modello CNN

procedo anche con data augmentation per prevenire overfitting aumentando rumore e nunerosità delle immagini 

In [ ]:

# Percorsi delle directory di origine e destinazione
source_dir_tsla = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/tsla'
source_dir_ko = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/ko'
augmented_dir_tsla = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/tsla_augmentated'
augmented_dir_ko = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/ko_augmentated'

# Creazione delle directory se non esistono
os.makedirs(augmented_dir_tsla, exist_ok=True)
os.makedirs(augmented_dir_ko, exist_ok=True)

# Funzione per applicare data augmentation e salvare le immagini aumentate
def augment_images(source_dir, augmented_dir, prefix, num_augmented_images=100):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    for img_name in os.listdir(source_dir):
        img_path = os.path.join(source_dir, img_name)
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=augmented_dir, save_prefix=prefix, save_format='png'):
            i += 1
            if i >= num_augmented_images:
                break

# Applicare data augmentation alle immagini di TSLA e KO
augment_images(source_dir_tsla, augmented_dir_tsla, prefix='TSLA')
augment_images(source_dir_ko, augmented_dir_ko, prefix='KO')


## trasfromazioni su spettrogrammi :....

costruzione del modello di Deep learning per la classificazione delle stock in base a volatilità/rischio-rendimento

In [6]:
import os
import sys
import platform

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler

import torchvision
from torchvision import transforms, datasets
import sklearn as sk
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

utilizziamo metal di apple per accelerare addestramento tramite l'utilizzo di gpu

In [7]:
has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

print("Stai utilizzando il dispositivo:", device)

Python Platform: macOS-14.4.1-arm64-arm-64bit
PyTorch Version: 2.3.1

Python 3.12.3 (v3.12.3:f6650f9ad7, Apr  9 2024, 08:18:47) [Clang 13.0.0 (clang-1300.0.29.30)]
Pandas 2.2.2
Scikit-Learn 1.4.2
NVIDIA/CUDA GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps
Stai utilizzando il dispositivo: mps


controllo 

In [8]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


train/test split

In [ ]:
train_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/augmentated'
test_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/test'

## Definisci i generatori di dati
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Carica le immagini di addestramento e test
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)


modello 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [4]:
# 1. Image dimensions and preprocessing
img_width, img_height = 224, 224  # Resize to a smaller shape for faster processing

train_data_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/train'
validation_data_dir = '/Users/roberto/Desktop/UNI 3^ ANNO/tesi/spettrogrammi/test'
batch_size = 32


In [5]:
# 2. Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Binary classification (KO vs TSLA)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 1894 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [6]:
# 3. Define the CNN Model
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Binary output (0 or 1)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# 4. Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# 5. Train the model
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


59/59 ━━━━━━━━━━━━━━━━━━━━ 52s 781ms/step - accuracy: 0.5148 - loss: 1.2842 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
 1/59 ━━━━━━━━━━━━━━━━━━━━ 28s 496ms/step - accuracy: 0.6250 - loss: 0.6920

2024-09-24 21:53:45.512948: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6250 - loss: 0.6920 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 49s 751ms/step - accuracy: 0.4944 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
 1/59 ━━━━━━━━━━━━━━━━━━━━ 28s 498ms/step - accuracy: 0.5000 - loss: 0.6931

...

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4375 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 50s 761ms/step - accuracy: 0.4704 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/10
 1/59 ━━━━━━━━━━━━━━━━━━━━ 28s 497ms/step - accuracy: 0.4062 - loss: 0.6936

2024-09-24 21:56:17.312343: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4062 - loss: 0.6936 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 9/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 49s 749ms/step - accuracy: 0.4900 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 10/10
 1/59 ━━━━━━━━━━━━━━━━━━━━ 28s 492ms/step - accuracy: 0.5625 - loss: 0.6928

2024-09-24 21:57:07.456992: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5625 - loss: 0.6928 - val_accuracy: 0.5000 - val_loss: 0.6931


In [9]:
# 6. Evaluate the model
scores = model.evaluate(validation_generator)
print(f"Validation Accuracy: {scores[1] * 100}%")

# Save the model for future use
model.save('stock_classification_model.h5')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.5000 - loss: 0.6931


Validation Accuracy: 50.0%
